In [ ]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

# Load the model
model = load_model('historyexp.keras')

In [ ]:
print(model)

In [ ]:
output_data = pd.read_csv('C:/Users/nvarshney014/OneDrive - PwC/Desktop/Project/Data/output.csv')

input_data = []
date_data = []

lengt = len(output_data["volt"])
last_one = []
for parameter in [  "volt" , "rotate", "pressure" , "vibration" ] :
    last_one += [ output_data[parameter][lengt-1] ]
input_data += [last_one]

date_data += [ output_data["date"][lengt-1] ]

date_data[0] = datetime.datetime.strptime(date_data[0] , "%d/%m/%Y")

In [ ]:
f = open("testing.csv","w")
ff = open("output.csv","r")

for line in ff.readlines() :
    f.write(line)
f.flush()

In [ ]:
scaler = StandardScaler()
features = ['volt','rotate','pressure','vibration']
output_data[features] = scaler.fit_transform(output_data[features])

label_encoder = LabelEncoder()
output_data['failure'] = label_encoder.fit_transform(output_data['failure'])

In [ ]:
def preprocess_input_data(input_data):
    print(input_data)
    input_data_reshaped = np.array(input_data).reshape(-1, len(features))
    input_data_scaled = scaler.transform(input_data_reshaped)  # Apply StandardScaler to the reshaped input_data
    input_data_final = input_data_scaled.reshape(1, seq_length, len(features))  # Reshape back to LSTM input shape
    return input_data_final



seq_length=2
machine_data={} # to store historical data for each machine id

while True: 
    machine_id=input("Enter Machine ID :")
    if machine_id not in machine_data:
         machine_data[machine_id]=[]


    cnt=0

    while True:
        test_data = input_data[len(input_data) - seq_length + 1 :]
        date_data = date_data[len(date_data) - seq_length + 1 :]

        print(f"Enter data for set {cnt+1}:")
        volt = float(input(f"Enter voltage for {cnt+1} step : "))
        rotation = float(input(f"Enter rotation for {cnt+1} step : "))
        pressure = float(input(f"Enter pressure for {cnt+1} step : "))
        vibration = float(input(f"Enter vibration for {cnt+1} step : "))
        test_data.append([volt, rotation, pressure, vibration])
        date_data.append(date_data[-1]+datetime.timedelta(days=1))
        input_data=test_data
        cnt=cnt+1

        input_data_preprocessed = preprocess_input_data(input_data)

        predictions_proba = model.predict(input_data_preprocessed)
        predictions_labelled = np.argmax(predictions_proba, axis=1)

        failure_found = -1 
        if predictions_labelled[0] == 0:  failure_found = 0
        else: failure_found = label_encoder.inverse_transform([predictions_labelled[0]])[0]
        

        f.write(  date_data[-1].strftime("%d/%m/%Y") + "," + machine_id + ","  + ",".join(list(map(str,test_data[-1]))) + "," + str(failure_found) + '\n')
        f.flush()

        value = predictions_labelled[0]
        print(input_data)
        if predictions_labelled[0] == 0: 
            print("No failure predicted. Continuing...\n")
            break
        else:
            failure_type = label_encoder.inverse_transform([predictions_labelled[0]])[0]
            print(f"Failure predicted: {failure_type}. Stopping...\n")
            

            sender_email = 'yourMailid'
            sender_password = 'yourPassword'
            receiver_emails = ['neelansh.varshney@pwc.com','neelanshvarshney06@gmail.com']
            subject = f"Failure Alert!! P{value-1} issue"
            body = f"""Concerned Team,
                \n
                The machine learning model of machine id {machine_id} has predicted a failure of level P{value-1}. Kindly check at the earliest.
                \n
                This is an auto-generated email. Please don't reply to it. Connect with the team who can handle it.
                \n
                Link to the Power BI report: https://app.powerbi.com/groups/me/reports/aa299b8a-e6e7-4518-8a3a-cb82029db690/ReportSection?experience=power-bi
                \n
                Machine Montoring Team,
                machinemonitoringteam@pwc.com
                PwC India
 
                """

            # Path to your Power BI file
            power_bi_file = r"C:\Users\nvarshney014\Desktop\Project\Data\Power BI\Machine_data 2.pbix"

            # Set up the SMTP server
            smtp_server = 'smtp.office365.com'  # Replace with your SMTP server address
            smtp_port = 587  # Replace with your SMTP port number
            server = smtplib.SMTP(smtp_server, smtp_port)
            server.starttls()
            server.login(sender_email, sender_password)

            # Create message container
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = ', '.join(receiver_emails)            
            msg['Subject'] = subject
            msg['Importance'] = 'high'  # Marking the email as important

            # Attach body of the email
            msg.attach(MIMEText(body, 'plain'))

            # Attach Power BI file
            # attachment = open(power_bi_file, 'rb')
            # part = MIMEBase('application', 'octet-stream')
            # part.set_payload((attachment).read())
            # encoders.encode_base64(part)
            # part.add_header('Content-Disposition', "attachment; filename= %s" % power_bi_file)
            # msg.attach(part)

            # Send the email
            server.sendmail(sender_email, receiver_emails, msg.as_string())
            server.quit()
            break


        

    print("End of program.")